<a href="https://colab.research.google.com/github/cjfghk5697/anomaly-detection-competition/blob/main/Add_Tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기록

## 07-16
Tune 기능 추가

RAM 메모리 문제로 못하는 중이다. 주말되면 memory limit 걸어볼 예정이다.

https://dacon.io/competitions/official/235894/overview/description

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd "/content/drive/MyDrive/input/"
#!unzip -q "/content/drive/MyDrive/input/train.zip" 

/content/drive/MyDrive/input


In [ ]:
#!pip3 install timm

In [ ]:
!pip install -U ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import warnings
warnings.filterwarnings('ignore')
from ray import tune
import torch.optim as optim

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
#import timm
import random
from typing import Tuple, Sequence, Callable
from sklearn.metrics import f1_score, accuracy_score

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
import time
from PIL import Image
import cv2

from torchvision.models import efficientnet_v2_s

device = torch.device('cuda')

In [ ]:
train_png = sorted(glob('./train/*.png'))

In [ ]:
train_y = pd.read_csv('./train_df.csv')

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [ ]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (512, 512))
    return img

In [ ]:
train_imgs = [img_load(m) for m in tqdm(train_png)]

100%|██████████| 4277/4277 [02:32<00:00, 28.13it/s]


In [ ]:
transforms_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])


In [ ]:
class Custom_dataset(Dataset):
    def __init__(self, 
                 img_paths, 
                 labels, 
                 mode='train',
                 transforms= Sequence[Callable]
            ) -> None:
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
        self.transforms = transforms

    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:
                img = img[::-1].copy()
            elif augmentation==2:
                img = img[:,::-1].copy()
        if self.mode=='test':
            pass
        img = Image.fromarray(img) # NumPy array to PIL image
        if self.transforms is not None:
            img = self.transforms(img)        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = efficientnet_v2_s(pretrained=True)
        self.dropout=nn.Dropout(p=0.2)
        self.SiLU=nn.SiLU(inplace=False)
        self.classifier = nn.Linear(1000, 88)
        nn.init.xavier_normal_(self.classifier.weight)

    def forward(self, x):
        x = self.model(x)
        x = self.dropout(x)
        x = self.SiLU(x) 
        x = self.classifier(x)

        return x

In [ ]:
batch_size = 2
epochs = 10

# Train
trainset  = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train', transforms=transforms_train)
lengths = [int(len(trainset)*0.8), int(len(trainset)*0.2)]


In [ ]:
lengths=[lengths[0],lengths[1]+1]
train_dataset, valid_dataset = torch.utils.data.random_split(trainset, lengths)

In [ ]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [ ]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
model = Network().to(device)

def train(config):

  optimizer = torch.optim.AdamW(model.parameters(), lr=config["lr"])
  criterion = nn.CrossEntropyLoss()

  scaler = torch.cuda.amp.GradScaler(enabled=True) 
  lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=0.001, last_epoch=-1)

  valid_loss_list=[]
  train_loss_list=[]
  for epoch in range(epochs):
      start=time.time()
      train_loss = 0
      train_pred=[]
      train_y=[]

      valid_loss = 0
      valid_pred=[]
      valid_y=[]
      acc=0

      model.train()
      for batch in (train_loader):
          optimizer.zero_grad()
          x = torch.tensor(batch[0], dtype=torch.float32, device=device)
          y = torch.tensor(batch[1], dtype=torch.long, device=device)
          with torch.cuda.amp.autocast():
              pred = model(x)
          loss = criterion(pred, y)


          scaler.scale(loss).backward()
          scaler.step(optimizer)
          scaler.update()
          
          train_loss += loss.item()/len(train_loader)
          train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
          train_y += y.detach().cpu().numpy().tolist()
      
      train_f1 = score_function(train_y, train_pred)


      TIME = time.time() - start
      lr_scheduler.step()
      acc+=train_f1
      tune.report(mean_accuracy=train_f1)


Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth


  0%|          | 0.00/82.7M [00:00<?, ?B/s]

In [ ]:
analysis = tune.run(
    train, config={"lr": tune.grid_search([0.001, 0.01, 0.1])})
df = analysis.dataframe()


2022-07-16 14:17:52,965	WARNING function_runner.py:604 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


In [ ]:
config = {
    "l1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}

result = tune.run(
    partial(train_cifar, data_dir=data_dir),
    resources_per_trial={"cpu": 8, "gpu": gpus_per_trial},
    config=config,
    num_samples=num_samples,
    scheduler=scheduler,
    progress_reporter=reporter,
    checkpoint_at_end=True)